In [2]:
from typing import Dict, Literal, Sequence

# Load 

In [ ]:
from dataclasses import dataclass
from typing import Dict

@dataclass
class DSGVOEntry:
    """Repräsentiert einen DSGVO-Eintrag mit hierarchischer Struktur"""
    kapitel_nr: int
    kapitel_name: str
    abschnitt_nr: int       # 0 falls nicht vorhanden
    abschnitt_name: str     # "" falls nicht vorhanden
    artikel_nr: int
    artikel_name: str
    absatz_nr: int          # 0 falls nicht explizit durch (#) angegeben
    unterabsatz_nr: int     # 0 falls nicht explizit durch (a), (b), (c), ... angegeben, ansonsten a=1, b=2, c=3, ...
    satz_nr: int            # 1 falls nicht explizit durch <sup>#</sup> angegeben
    text: str

    def to_dict(self) -> Dict:
        """Konvertiert den Eintrag zu einem Dictionary für JSONL-Export"""
        return {
            'Kapitel_Nr': self.kapitel_nr,
            'Kapitel_Name': self.kapitel_name,
            'Abschnitt_Nr': self.abschnitt_nr,
            'Abschnitt_Name': self.abschnitt_name,
            'Artikel_nr': self.artikel_nr,
            'Artikel_Name': self.artikel_name,
            'Absatz_nr': self.absatz_nr,
            'Unterabsatz_nr': self.unterabsatz_nr,
            'Satz_nr': self.satz_nr,
            'Text': self.text
        }

In [24]:
import json

dsgvo_entries = []

def load_dsgvo_data():
    with open("../data/output/dsgvo_crawled_2025-08-11_1804.jsonl", "r") as f:
        for line in f:
            data = json.loads(line)
            dsgvo_entry = DSGVOEntry(
                kapitel_nr=data["Kapitel_Nr"],
                kapitel_name=data["Kapitel_Name"],
                abschnitt_nr=data["Abschnitt_Nr"],
                abschnitt_name=data["Abschnitt_Name"],
                artikel_nr=data["Artikel_nr"],
                artikel_name=data["Artikel_Name"],
                absatz_nr=data["Absatz_nr"],
                unterabsatz_nr=data["Unterabsatz_nr"],
                satz_nr=data["Satz_nr"],
                text=data["Text"]
            )
            dsgvo_entries.append(dsgvo_entry)

load_dsgvo_data()

In [4]:
def generate_normal_questions(
    batch: Sequence, prompt_type: Literal["question", "imperative"], open_ai_client
) -> Sequence:
    """Generate questions for context."""
    results = []
    _prompt = prompts[prompt_type]
    for doc in batch:
        llm_response: OpenAiChatResult = open_ai_client(
            prompt=_prompt.format(context=doc["context"]),
            completion_kwargs={"temperature": random.uniform(0.0, 1.0), "max_tokens": 750},  # noqa: S311
        )
        result = {
            "_key": doc["_key"],
            f"meta_data_{prompt_type}": meta_data_factory(),
            f"llm_response_{prompt_type}": asdict(llm_response),
        }
        results.append(result)
        print(result)
        print("---")
        print(llm_response.content)
        print()
    return results

In [32]:
import os

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
expected_output_schema = {
	"type": "object",
    "properties": {
		"questions": {
			"type": "array",
			"items": { "type": "string" }
		}
	},
    "required": ["questions"],
    "additionalProperties": False
}

In [ ]:
def create_normal_questions(dsgvo_entry: DSGVOEntry):
    response = client.responses.create(
	    model = "gpt-5-nano",
        input = [
		    {
			    "role": "user",
			    "content": """
Erstelle 6 Fragen in Deutscher Sprache zu folgendem Text.
Es muss möglich sein die Frage anhand des Textes zu beantworten.
Die Fragen sollen nicht die Wörter "und" enthalten.

Hier der Text:

{dsgvo_entry.text}

Die Fragen:

"""
		    }
	    ],
        text = {
			"format": {
				"type": "json_schema",
				"name": "question_item",
				"schema": expected_output_schema,
				"strict": True
			}
    	}
    )
    return response

In [48]:
for dsgvo_entry in dsgvo_entries:
    response = create_normal_questions(dsgvo_entry)
    questions = json.loads(response.output_text)["questions"]
    for question in questions:
        print(question)
    break

Was wird im Text als Zweck der Verarbeitung genannt?
Welche Datenarten werden im Text erfasst?
Welche Rechtsgrundlage für die Verarbeitung wird im Text genannt?
Welche Speicherdauer der Verarbeitung wird im Text genannt?
Welche Rechte der betroffenen Personen werden im Text genannt?
Welche technischen oder organisatorischen Maßnahmen zur Sicherheit der Verarbeitung werden im Text beschrieben?
